# Grid Fluid

Modeling fluid motion by solving the incompressible and inviscid Navier-Stokes equation on a uniform grid

Credit to Bo Zhu, on whose Dartmouth COSC89 course notes the following code has been based.

Additionally, credit to Robert Bridson from the University of British Columbia, whose Fluid Simulation Course Notes were invaluable in understanding the material. His course notes can be found here:\
https://www.cs.ubc.ca/~rbridson/fluidsimulation/fluids_notes.pdf

In [2]:
import taichi as ti
import taichi.math as tm
import math

ti.init()

[Taichi] version 1.1.2, llvm 10.0.0, commit f25cf4a2, osx, python 3.9.7
[I 10/27/22 15:39:27.860 1443210] [shell.py:_shell_pop_print@33] Graphical python shell detected, using wrapped sys.stdout
[Taichi] Starting on arch=x64


In [11]:
### Global Variables ###

substeps = 10
iters = 20
dt = 0.02 
res = 500
num_nodes = res * res
dx = 0.015

vel = ti.Vector.field(2, float, shape=(res, res)) # do I make the grid with less nodes?
new_vel = ti.Vector.field(2, float, shape=(res, res))
vel_divs = ti.field(float, shape=(res, res))
vel_curls = ti.field(float, shape=(res, res))
pressure = ti.field(float, shape=(res, res))
new_pressure = ti.field(float, shape=(res, res))
smoke_density = ti.field(float, shape=(res,res))
vor = ti.field(gloat, shape=(res,res))

src_pos = tm.vec2(0.5, 0.5)
src_vel = tm.vec2(1.5, 0)
src_rad = 0.1f

debug = False

In [ ]:
### Helper Functions for Projection ###

# @param - coordinates (i,j) of the node
# @return - gradient of p at node (i,j)
@ti.func 
def gradient_pressure(i:ti.i32, j:ti.i32):
    pressures = tm.vec2(pressure[i+1, j] - p[i-1, j], p[i, j+1] - p[i, j-1])
    return pressures / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - divergence of the velocity at grid node (i, j)
@ti.func
def divergence_vel(i:ti.i32, j:ti.i32):
    return (vel[i+1, j][0] - vel[i-1, j][0] + vel[i, j+1][1] - vel[i, j-1][1]) / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - the curl of the velocity at grid node (i, j)
@ti.func
def curl_vel():
    return (vel[i+1, j][1] - vel[i-1, j][1] + vel[i, j+1][0] - vel[i, j-1][0]) / (2 * dx)

# @param - coordinates (i,j) of the grid node
# @return - the laplacian of the pressure at grid node (i, j)
@ti.func
def laplacian_pressure():
    return (pressure[i+1, j] + pressure[i-1, j] + pressure[i, j+1] + pressure[i, j-1] - 4 * pressure[i, j]) / (dx * dx)

In [8]:
### Simulator Functions ###

# @param - x_p: vec2, where x_p = (x - u^(n+1/2)(x) * dt) (the "starting point")
# @return - the interpolated value for u*(x) at x_p
@ti.func
def interpolate(x_p):
    x = x_p[0]
    y = x_p[1]
    x1 = math.floor(x_p[0])
    x2 = math.ceil(x_p[0])
    y1 = math.floor(x_p[1])
    y2 = math.ceil(x_p[1])
    # interpolate in x direction
    f_xy1 = (x2 - x)/(x2-x1) * vel[x1, y1] + (x - x1)/(x2 - x1) * vel[x2, y1]
    f_xy2 = (x2 - x)/(x2-x1) * vel[x1, y2] + (x - x1)/(x2 - x1) * vel[x2, y2]
    # interpolate in y direction
    f_xy = (y2 - y)/(y2 - y1) * f_xy1 + (y - y1)/(y2 - y1) * f_xy2
    return f_xy

# same thing as function above, but uses the density matrix instead
@ti.func
def interpolate_density(x_p):
    x = x_p[0]
    y = x_p[1]
    x1 = math.floor(x_p[0])
    x2 = math.ceil(x_p[0])
    y1 = math.floor(x_p[1])
    y2 = math.ceil(x_p[1])
    # interpolate in x direction
    f_xy1 = (x2 - x)/(x2-x1) * smoke_density[x1, y1] + (x - x1)/(x2 - x1) * smoke_density[x2, y1]
    f_xy2 = (x2 - x)/(x2-x1) * smoke_density[x1, y2] + (x - x1)/(x2 - x1) * smoke_density[x2, y2]
    # interpolate in y direction
    f_xy = (y2 - y)/(y2 - y1) * f_xy1 + (y - y1)/(y2 - y1) * f_xy2
    return f_xy

# Advection using a semi-Lagrangian method
@ti.kernel
def advect():
#     for row in range(res):
#         for col in range(res):
    for i in ti.grouped(vel):
        pos = tm.vec2(i)
        new_pos = pos - vel[i] * 0.5 * dt
        new_vel = interpolate(new_pos)
        new_pos = pos - new_vel * dt
        new_vel = interpolate(new_pos)
        vel[i] = new_vel
        new_den = interpolate_density(new_pos)
        smoke_density[i] = new_den

@ti.kernel
def project():
    # Calc velocity for RHS of Poissan (div u*)
    # Update the pressure for each grid node iteratively multiple times
    # Update new_velocity
    pass

@ti.func
def clear_vorticity():
    pass

@ti.kernel
def vorticity_confinement():
    # find vorticity confinement force
    # update velocity for each point
    pass

# create a "source" for the smoke
@ti.kernel
def source():
    # iterate over each grid node
    # if it's within "source_radius" of "source_pos", vel at that node is src_vel, smoke_density there is 1
    pass

In [13]:
def substep():
    # source
    # swap new_velocity and velocity to update the n+1 as n for this time step
    # advect
    # project
    # vorticity confinement
    pass

In [16]:
### Setup ###
@ti.kernel
def init_grid():
    pass

In [18]:
### Driver ###

def main():
    gui = ti.GUI('Grid Fluid', (res, res))
        
    init_grid()

    if debug:
        print("got here!")
        substep()
        
    while gui.running:
        
        # Move stuff
        if not debug:
            for step in range(substeps): # do I need this?
                substep()
             
        # Draw grid

        
        gui.show()
    
if __name__ == '__main__':
    main()